In [20]:
import numpy as np
import os
import pandas as pd

import os
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from random import sample

In [21]:
train_dir = 'images'
dfPath = 'train.csv'

df = pd.read_csv(dfPath)
df = df.astype(str)

In [22]:
amt = df.groupby(['hotel_id'])['hotel_id'].count().reset_index(name="counts")
amt.columns = ["hotel","county"]
amt = amt[amt["county"] < 10]

idxesToDelete = []
for idx, rowHotelCat in amt.iterrows():
    hotelCat = rowHotelCat["hotel"]
    
    #print(hotelCat)
    #print(df[df["hotel_id"] == hotelCat].index)
    
    df.drop(df[df["hotel_id"] == hotelCat].index, inplace = True)

newAmt = df.groupby(['hotel_id'])['hotel_id'].count().reset_index(name="counts")
newAmt.columns = ["hotel","county"]
print(newAmt)

       hotel  county
0     100206      17
1     100297      19
2      10031      16
3     100326      20
4     100410      19
...      ...     ...
1371   99038      22
1372    9906      16
1373   99234      16
1374   99311      11
1375    9991      14

[1376 rows x 2 columns]


In [23]:
df.to_csv("new.csv")

In [25]:
import random
train = df
valid = pd.DataFrame()

proportion = 0.2

for idx, rowHotelCat in newAmt.iterrows():
    hotelCat = rowHotelCat["hotel"]
    amount = rowHotelCat["county"]
    target = int(amount * proportion)
    
    chosenIdx = list(df[df["hotel_id"] == hotelCat].index.values)
    toMove = random.sample(chosenIdx, target)
    
    rowsToMove = df.loc[toMove]
    
    train.drop(toMove, inplace = True)
    valid = pd.concat([valid, rowsToMove], ignore_index = True)
    
    for i,row in rowsToMove.iterrows():
        os.rename(os.path.join(train_dir,row["image_id"]),os.path.join('test',row["image_id"]))

train.to_csv("trainSplit.csv")
valid.to_csv("validSplit.csv")

In [28]:
print(len(os.listdir("images")))
print(len(os.listdir("test")))

38526
6178


In [13]:
import random

targetMax = 75
for idx, rowHotelCat in newAmt.iterrows():
    hotelCat = rowHotelCat["hotel"]
    amount = rowHotelCat["county"]
    if amount > targetMax:
        chosenIdx = list(df[df["hotel_id"] == hotelCat].index.values)
        toDelete = random.sample(chosenIdx, amount - targetMax)
        df.drop(toDelete, inplace = True)

newAmt = df.groupby(['hotel_id'])['hotel_id'].count().reset_index(name="counts")
newAmt.columns = ["hotel","county"]
#print(newAmt)

In [14]:
df.to_csv("withoutBiggest.csv")

In [19]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [20]:
overSampled = df
target = 30

for idx, rowHotelCat in newAmt.iterrows():
    hotelCat = rowHotelCat["hotel"]
    amount = rowHotelCat["county"]
    
    imagesInCat = list(df[df["hotel_id"] == hotelCat].index.values)
    amount = len(imagesInCat)
    nToPrint = target - amount
    nToPrint = min(amount,nToPrint)
    
    if int(nToPrint) > 0:
        toAugment = random.sample(imagesInCat, nToPrint)
        made = 0
        print("Making", nToPrint,"for:", hotelCat)
        
        for currIdxImage in toAugment:
            if made < nToPrint:
                img_path = df.loc[currIdxImage]["image_id"]
                img_path = os.path.join(train_dir, img_path)
                img = tf.keras.preprocessing.image.load_img(img_path)
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                img_array = img_array.reshape((1,) + img_array.shape)

                for batch in train_datagen.flow(img_array, batch_size=1):
                    temp = tf.keras.preprocessing.image.array_to_img(batch[0])
                    name = hotelCat + 'x'+ str(made) + '.jpg'
                    nameTwo = os.path.join(train_dir, name)
                    temp.save(nameTwo)
                    made += 1
                    row = pd.DataFrame({"image_id" : name, "hotel_id": hotelCat}, index=[0])
                    overSampled = pd.concat([overSampled, row], ignore_index = True)
                    break

                if made >= nToPrint:
                    break

Making 13 for: 100206
Making 11 for: 100297
Making 14 for: 10031
Making 10 for: 100326
Making 11 for: 100410
Making 14 for: 10054
Making 12 for: 100784
Making 10 for: 1009
Making 10 for: 101899
Making 14 for: 102119
Making 2 for: 102219
Making 14 for: 10290
Making 10 for: 102995
Making 14 for: 103319
Making 8 for: 103379
Making 15 for: 103533
Making 8 for: 103650
Making 10 for: 10389
Making 12 for: 10411
Making 12 for: 10503
Making 11 for: 10578
Making 10 for: 10678
Making 9 for: 10684
Making 8 for: 107043
Making 12 for: 107115
Making 11 for: 10766
Making 12 for: 10785
Making 3 for: 10849
Making 12 for: 108817
Making 11 for: 10884
Making 11 for: 10959
Making 6 for: 110119
Making 12 for: 11025
Making 11 for: 11039
Making 15 for: 110509
Making 11 for: 11060
Making 3 for: 11099
Making 15 for: 11101
Making 11 for: 11165
Making 6 for: 11200
Making 12 for: 11257
Making 13 for: 11289
Making 14 for: 11311
Making 12 for: 113206
Making 14 for: 1144
Making 12 for: 114886
Making 14 for: 115137
Mak

In [22]:
print(overSampled)
overSampled.to_csv("oversampled.csv")

            image_id hotel_id
0      000011648.jpg    15526
1      000011630.jpg    15526
2      000011650.jpg    15526
3      000011633.jpg    15526
4      000011656.jpg    15526
...              ...      ...
40586     9991x9.jpg     9991
40587    9991x10.jpg     9991
40588    9991x11.jpg     9991
40589    9991x12.jpg     9991
40590    9991x13.jpg     9991

[40591 rows x 2 columns]
